# Community Keyword Extractor

In [1]:
!pip install keybert

In [27]:
!pip install tqdm

In [2]:
!python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/sallycaoyu/Downloads/anaconda3/lib/python3.7/site-packages/en_core_web_sm
-->
/Users/sallycaoyu/Downloads/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [38]:
import pickle
import re
import numpy as np
import pandas as pd
import spacy
import keybert
from collections import defaultdict, Counter
from keybert import KeyBERT
from tqdm import tqdm, tqdm_pandas

In [19]:
with open('nodes_subreddit_1000.csv', 'rb') as file:
    data = pd.read_csv(file)
print(data.shape)
data.head()

(654, 3)


,subreddit,subreddit_feat1,words
0,gaming,1,"**If your submission does not appear, do not d..."
1,psychology,3,###We now have user flair! Please message the ...
2,DIY,6,[](#/RES_SR_Config/NightModeCompatible)\n\n> #...
3,Justrolledintotheshop,8,For those absolutely stupid things that you se...
4,tattoos,6,"# Hey there, welcome to /r/tattoos. Please re..."


In [42]:
kw_model = KeyBERT()
progress_bar = tqdm.pandas()

In [48]:
def community_keyword_extractor(words):
    '''extract community keyword using KeyBERT'''
    keyword_scores = kw_model.extract_keywords(words, keyphrase_ngram_range=(1, 1), stop_words='english',
                                         use_maxsum=True, nr_candidates=10, top_n=5)

    return '|'.join([kw for kw, score in keyword_scores])

In [49]:
data['genre'] = data['words'].progress_apply(lambda x: community_keyword_extractor(x))
data

100%|██████████| 654/654 [04:22<00:00,  2.49it/s]


,subreddit,subreddit_feat1,words,genre
0,gaming,1,"**If your submission does not appear, do not d...",ban|content|moderators|removed|submit
1,psychology,3,###We now have user flair! Please message the ...,psychopharmacology|research|behavioralmedicine...
2,DIY,6,[](#/RES_SR_Config/NightModeCompatible)\n\n> #...,chat|im_the_blondihacks_home_shop_machinist_yo...
3,Justrolledintotheshop,8,For those absolutely stupid things that you se...,automotivetraining|submissions|troubleshooting...
4,tattoos,6,"# Hey there, welcome to /r/tattoos. Please re...",artists|emojis|guidelines|ive_got_a_bit_of_a_r...
5,3DS,1,###[Mario & Luigi: Bowser's Inside Story + Bow...,gamefaqs|nintendoswitch|luigi|23r3ds|3dsdeals
6,bestof,8,#This subreddit features the very best comment...,newreddits|guidelines|removed|restrict_sr|best...
7,Foodforthought,2,# Intelligent and thought-provoking commentari...,reddit|intellectual|editorialize|ban|discourse
8,4chan,2,----\n\n##Community: \n\n**Posts:** /[new](ht...,week|4chanexploitables|month|www|gilded
9,Art,3,**Welcome to /r/art!** *This is a community of...,whatisthispainting|title|artstore|share_your_a...
